In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pyspark.sql.functions as F
import pyspark.sql.types as T
from pyspark.sql import SparkSession

In [3]:
from shared.schema import DatasetSchema

DATASET = DatasetSchema.load_schema('DBLP-V3')
DATASET.save_schema()

In [4]:
spark = (SparkSession.builder
         .appName(f'{DATASET}')
         .config('spark.sql.legacy.timeParserPolicy', 'LEGACY')
         .config("spark.executor.memory", "8g")
         .config("spark.driver.memory", "8g")
         .config("spark.memory.offHeap.enabled", True)
         .config("spark.memory.offHeap.size", "16g")
         .getOrCreate())

22/01/23 01:31:24 WARN Utils: Your hostname, megatron resolves to a loopback address: 127.0.1.1; using 192.168.1.89 instead (on interface enp7s0)
22/01/23 01:31:24 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/01/23 01:31:25 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/01/23 01:31:26 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
22/01/23 01:31:26 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.
22/01/23 01:31:26 WARN Utils: Service 'SparkUI' could not bind on port 4042. Attempting port 4043.
22/01/23 01:31:26 WARN Utils: Service 'SparkUI' could not bind on port 4043. Attempting port 4044.


In [5]:
df = (
    spark.read.text(DATASET.raw_str('DBLPOnlyCitationOct19.txt'), wholetext=False, lineSep='\n\n')
        .withColumn('value', F.split(F.col('value'), '\n'))
)

df.head(10)

[Row(value=['1632442', '#*OQL[C++]: Extending C++ with an Object Query Capability.', '#@José A. Blakeley', '#t1995', '#cModern Database Systems', '#index0']),
 Row(value=['#*Transaction Management in Multidatabase Systems.', '#@Yuri Breitbart,Hector Garcia-Molina,Abraham Silberschatz', '#t1995', '#cModern Database Systems', '#index1']),
 Row(value=['#*Overview of the ADDS System.', '#@Yuri Breitbart,Tom C. Reyes', '#t1995', '#cModern Database Systems', '#index2']),
 Row(value=['#*Multimedia Information Systems: Issues and Approaches.', '#@Stavros Christodoulakis,Leonidas Koveos', '#t1995', '#cModern Database Systems', '#index3']),
 Row(value=['#*Active Database Systems.', '#@Umeshwar Dayal,Eric N. Hanson,Jennifer Widom', '#t1995', '#cModern Database Systems', '#index4', '#%995520']),
 Row(value=['#*Where Object-Oriented DBMSs Should Do Better: A Critique Based on Early Experiences.', '#@Angelika Kotz Dittrich,Klaus R. Dittrich', '#t1995', '#cModern Database Systems', '#index5']),
 Row(

In [6]:
schema = T.StructType([
    T.StructField('title', T.StringType(), True),
    T.StructField('authors', T.ArrayType(T.StringType(), False), False),
    T.StructField('year', T.IntegerType(), True),
    T.StructField('venue', T.StringType(), True),
    T.StructField('index', T.StringType(), True),
    T.StructField('references', T.ArrayType(T.StringType(), False), False),
    T.StructField('abstract', T.StringType(), True),
])

@F.udf(returnType=schema)
def parse_citation(lines):
    result = {
        'title': None,
        'authors': [],
        'year': None,
        'venue': None,
        'index': None,
        'references': [],
        'abstract': None,
    }
    for line in lines:
        if line.startswith('#*'):
            result['title'] = line[2:].strip()
        elif line.startswith('#@'):
            result['authors'].extend(line[2:].strip().split(','))
        elif line.startswith('#t'):
            result['year'] = int(line[2:].strip())
        elif line.startswith('#c'):
            result['venue'] = line[2:].strip()
        elif line.startswith('#index'):
            result['index'] = int(line[6:].strip())
        elif line.startswith('#%'):
            result['references'].extend(line[2:].strip().split(','))
        elif line.startswith('#!'):
            result['abstract'] = line[2:].strip()
    return result

df_papers = df.select(
    parse_citation(F.col('value')).alias('parsed_citation')
).select('parsed_citation.*').cache()
df_papers.head(5)

[Row(title='OQL[C++]: Extending C++ with an Object Query Capability.', authors=['José A. Blakeley'], year=1995, venue='Modern Database Systems', index='0', references=[], abstract=None),
 Row(title='Transaction Management in Multidatabase Systems.', authors=['Yuri Breitbart', 'Hector Garcia-Molina', 'Abraham Silberschatz'], year=1995, venue='Modern Database Systems', index='1', references=[], abstract=None),
 Row(title='Overview of the ADDS System.', authors=['Yuri Breitbart', 'Tom C. Reyes'], year=1995, venue='Modern Database Systems', index='2', references=[], abstract=None),
 Row(title='Multimedia Information Systems: Issues and Approaches.', authors=['Stavros Christodoulakis', 'Leonidas Koveos'], year=1995, venue='Modern Database Systems', index='3', references=[], abstract=None),
 Row(title='Active Database Systems.', authors=['Umeshwar Dayal', 'Eric N. Hanson', 'Jennifer Widom'], year=1995, venue='Modern Database Systems', index='4', references=['995520'], abstract=None)]

In [7]:
df_nodes_authors = (
    df_papers.select(
        F.explode(F.col('authors')).alias('name'),
    ).withColumn('id', F.col('name')).filter(F.col('id').isNotNull())
        .dropDuplicates(['id'])
)
print(df_nodes_authors.count())
df_nodes_authors.head(5)

1036991


[Row(name=' Aihua Bao', id=' Aihua Bao'),
 Row(name=' C.D.', id=' C.D.'),
 Row(name=' D.F.', id=' D.F.'),
 Row(name=' F.F.', id=' F.F.'),
 Row(name=' G.A.', id=' G.A.')]

In [8]:
df_nodes_venues = (
    df_papers.select(
        F.col('venue').alias('id'),
        F.col('venue').alias('name'),
    ).filter("id != ''").filter(F.col('id').isNotNull())
        .dropDuplicates(['id'])
)
print(df_nodes_venues.count())
df_nodes_venues.head(5)

7707


[Row(id='10th Anniversary Colloquium of UNU/IIST', name='10th Anniversary Colloquium of UNU/IIST'),
 Row(id='13th Annual Symposium on Switching and Automata Theory', name='13th Annual Symposium on Switching and Automata Theory'),
 Row(id='14th Annual Symposium on Switching and Automata Theory', name='14th Annual Symposium on Switching and Automata Theory'),
 Row(id='15. WLP', name='15. WLP'),
 Row(id='15th Annual Symposium on Switching and Automata Theory', name='15th Annual Symposium on Switching and Automata Theory')]

In [9]:
df_nodes_papers = (
    df_papers.select(
        F.col('index').alias('id'),
        'title',
        'authors',
        'venue',
        'year',
        'abstract',
        F.to_timestamp(F.col('year').cast('string'), 'yyyy').alias('timestamp')
    ).filter(F.col('id').isNotNull())
        .dropDuplicates(['id'])
)
print(df_nodes_papers.count())
df_nodes_papers.head(5)

1632442


[Row(id='1000015', title='Some strongly undecidable natural arithmetical problems, with an application to intuitionistic theories.', authors=['Panu Raatikainen'], venue='J. Symb. Log.', year=2003, abstract=None, timestamp=datetime.datetime(2003, 1, 1, 0, 0)),
 Row(id='1000067', title='Two Interpolation Theorems for a pi Predicate Calculus.', authors=['Shoji Maehara', 'Gaisi Takeuti'], venue='J. Symb. Log.', year=1971, abstract=None, timestamp=datetime.datetime(1971, 1, 1, 0, 0)),
 Row(id='1000073', title='Compactness and Transfer for a Fragment of L.', authors=['Menachem Magidor', 'Jerome I. Malitz'], venue='J. Symb. Log.', year=1977, abstract=None, timestamp=datetime.datetime(1977, 1, 1, 0, 0)),
 Row(id='1000087', title='An Example Concerning Scott Heights.', authors=['Michael Makkai'], venue='J. Symb. Log.', year=1981, abstract=None, timestamp=datetime.datetime(1981, 1, 1, 0, 0)),
 Row(id='1000096', title='The Deduction Theorem for Quantum Logic - Some Negative Results.', authors=['J

In [10]:
df_node_ids = (
    df_nodes_authors.select('id')
        .union(df_nodes_venues.select('id'))
        .union(df_nodes_papers.select('id'))
        .distinct()
)

def filter_node_ids(df):
    return df.join(
        df_node_ids,
        F.col('src') == F.col('id'),
        'inner'
    ).drop(
        'id'
    ).join(
        df_node_ids,
        F.col('dst') == F.col('id'),
        'inner'
    ).drop('id')

In [11]:
df_edges_authored = filter_node_ids(
    df_papers.select(
        F.explode(F.col('authors')).alias('src'),
        F.col('index').alias('dst'),
    ).distinct()
)
print(df_edges_authored.count())
df_edges_authored.head(5)

4285548


[Row(src='Panu Raatikainen', dst='1000015'),
 Row(src='Shoji Maehara', dst='1000067'),
 Row(src='Gaisi Takeuti', dst='1000067'),
 Row(src='Jerome I. Malitz', dst='1000073'),
 Row(src='Menachem Magidor', dst='1000073')]

In [12]:
df_edges_published_in = filter_node_ids(
    df_papers.select(
        F.col('index').alias('src'),
        F.col('venue').alias('dst'),
    ).filter("dst != ''").distinct()
)
print(df_edges_published_in.count())
df_edges_published_in.head(5)

1630753


[Row(src='1203588', dst='ACISP'),
 Row(src='1203604', dst='ACISP'),
 Row(src='1509815', dst='ACISP'),
 Row(src='1509827', dst='ACISP'),
 Row(src='18947', dst='ACISP')]

In [13]:
df_edges_cited = filter_node_ids(
    df_papers.select(
        F.col('index').alias('src'),
        F.explode(F.col('references')).alias('dst'),
    )
        .withColumn('dst', F.col('dst').cast('int').cast('string'))
        .distinct()

)
print(df_edges_cited.count())
df_edges_cited.head(5)

2327450


[Row(src='868396', dst='1000725'),
 Row(src='1093654', dst='1000725'),
 Row(src='1117139', dst='1001246'),
 Row(src='1598187', dst='1001246'),
 Row(src='55859', dst='1001246')]

In [14]:
df_nodes_authors.write.parquet(DATASET.processed_str('nodes_Author'), mode='overwrite')
df_nodes_venues.write.parquet(DATASET.processed_str('nodes_Venue'), mode='overwrite')
df_nodes_papers.write.parquet(DATASET.processed_str('nodes_Paper'), mode='overwrite')

df_edges_authored.write.parquet(DATASET.processed_str('edges_AUTHORED'), mode='overwrite')
df_edges_published_in.write.parquet(DATASET.processed_str('edges_PUBLISHED_IN'), mode='overwrite')
df_edges_cited.write.parquet(DATASET.processed_str('edges_CITED'), mode='overwrite')

In [15]:
from shared.schema.graph import GraphSchema, NodeSchema, EdgeSchema

(
    GraphSchema()
        .add_node_schema('Author', NodeSchema.from_spark(df_nodes_authors.schema, label='name'))
        .add_node_schema('Venue', NodeSchema.from_spark(df_nodes_venues.schema, label='name'))
        .add_node_schema('Paper', NodeSchema.from_spark(df_nodes_papers.schema, label='title', timestamp='timestamp', interaction=False))
        .add_edge_schema('AUTHORED', EdgeSchema.from_spark(df_edges_authored.schema, source_type='Author', target_type='Paper', directed=True))
        .add_edge_schema('PUBLISHED_IN', EdgeSchema.from_spark(df_edges_published_in.schema, source_type='Paper', target_type='Venue', directed=True))
        .add_edge_schema('CITED', EdgeSchema.from_spark(df_edges_cited.schema, source_type='Paper', target_type='Paper', directed=True))
        .save_schema(DATASET.processed())
)

GraphSchema(_path=PosixPath('/dd_volume/Development/Python/Thesis/code/datasets/data/processed/DBLP-V3'), nodes={'Author': NodeSchema(_type='Author', _schema=..., label='name', properties={'name': GraphProperty(_name='name', dtype=DType(atomic=<DTypeAtomic.STRING: 'string'>, array=False)), 'id': GraphProperty(_name='id', dtype=DType(atomic=<DTypeAtomic.STRING: 'string'>, array=False))}, dynamic=None), 'Venue': NodeSchema(_type='Venue', _schema=..., label='name', properties={'id': GraphProperty(_name='id', dtype=DType(atomic=<DTypeAtomic.STRING: 'string'>, array=False)), 'name': GraphProperty(_name='name', dtype=DType(atomic=<DTypeAtomic.STRING: 'string'>, array=False))}, dynamic=None), 'Paper': NodeSchema(_type='Paper', _schema=..., label='title', properties={'id': GraphProperty(_name='id', dtype=DType(atomic=<DTypeAtomic.STRING: 'string'>, array=False)), 'title': GraphProperty(_name='title', dtype=DType(atomic=<DTypeAtomic.STRING: 'string'>, array=False)), 'authors': GraphProperty(_na

# Ground truth communities
Using same methodology as in:

J. Yang and J. Leskovec, “Defining and evaluating network communities based on ground-truth,” in Proceedings of the ACM SIGKDD Workshop on Mining Data Semantics, New York, NY, USA, Aug. 2012, pp. 1–8. doi: 10.1145/2350190.2350193.

In [16]:
print(df_nodes_papers.groupby('venue').count().head(100))
print(df_nodes_papers.filter(F.col('venue').isNull()).head(5))

[Row(venue='QoSA', count=76), Row(venue='Empirical Software Engineering', count=302), Row(venue='CSR', count=223), Row(venue='GIS', count=368), Row(venue='International Conference on Internet Computing', count=847), Row(venue='ICECCS', count=570), Row(venue='Formal Asp. Comput.', count=573), Row(venue='Parts, Hybrids, and Packaging, IEEE Transactions on', count=342), Row(venue='Intelligent Robots', count=27), Row(venue='Intelligent Control, 2002. Proceedings of the 2002 IEEE International Symposium on', count=158), Row(venue='Multimedia Information Retrieval', count=341), Row(venue='PAKDD (2)', count=49), Row(venue='WiOpt', count=169), Row(venue='EASSS', count=21), Row(venue='ECAI Workshop on Ontology Learning', count=13), Row(venue='KWEPSY', count=31), Row(venue='Handbook on Ontologies', count=33), Row(venue='Designing Smart Homes', count=11), Row(venue='ICPR (3)', count=796), Row(venue='BIOKDD', count=52), Row(venue='Intelligent Search on XML Data', count=20), Row(venue='GRAPP (GM/R)

[]


In [17]:
df_nodes_papers.groupby('venue').count().filter('count < 10').count()

1148

In [18]:
df_nodes_papers.groupby('venue').count().filter('count > 100').head(5)

[Row(venue='Empirical Software Engineering', count=302),
 Row(venue='CSR', count=223),
 Row(venue='GIS', count=368),
 Row(venue='International Conference on Internet Computing', count=847),
 Row(venue='ICECCS', count=570)]

## Cant compute ground truth communities. Venue is actually a journal